In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002216339111328125
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 41.237321043001295
200000 76.19512705864327
300000 95.1527715587012
400000 99.06071297122374
500000 98.01841529886251
600000 101.07512538522488
700000 98.17099400958197
800000 95.89053966722817
900000 96.36567385468341
1000000 97.73711532328579
1100000 95.62081051457925
1200000 25.681736984484584
1300000 95.67085670595554
1400000 96.30394660428676
1500000 94.83437372108035
1600000 94.52749066445782
1700000 95.8814291664433
1800000 100.7321755076079
1900000 96.70318467813385
2000000 95.5903904980796
2100000 96.14889715525443
2200000 96.75959949255393
2300000 74.25688797590908
2400000 45.838547767184686
2500000 91.23944076656198
2600000 90.8993691341881
2700000 90.96970379383431
2800000 89.8962776763996
2900000 93.5753295590838
3000000 91.38498134131181
3

29800000 75.77065970803399
29900000 107.32062840915907
30000000 101.10331059961669
30100000 99.18269382908906
30200000 41.924503048814394
30300000 102.12654164262506
30400000 102.69376253166467
30500000 100.35877226577794
30600000 97.55918376032565
30700000 99.1782988360619
30800000 96.03450308859871
30900000 95.99326314510503
31000000 94.46184666714389
31100000 34.788626968737326
31200000 113.08129376980727
31300000 92.78088066239617
31400000 95.20212602273072
31500000 98.59723803008767
31600000 99.63923538299652
31700000 98.55213065375081
31800000 101.11201505164843
31900000 98.92845021064639
32000000 57.24018747996811
32100000 87.76940653771078
32200000 103.36775397266487
32300000 105.20206605434312
32400000 108.69384077506089
32500000 35.16240750931238
32600000 107.85906814306385
32700000 107.7973874828421
32800000 104.9349335887728
32900000 105.62084701433876
33000000 103.6427948307774
33100000 100.96560853098566
33200000 98.63003434744795
33300000 94.48042769404626
33400000 26.83

60000000 100.42003744698539
60100000 102.73014685092548
60200000 102.59433913264412
60300000 36.86337627068841
60400000 97.18317207442122
60500000 94.76602094893502
60600000 95.0924362907326
60700000 91.93400963312969
60800000 89.07645148425671
60900000 90.11781707872058
61000000 23.33490267663762
61100000 94.19560566212813
61200000 91.66299396532828
61300000 90.62495044194199
61400000 88.70924328060633
61500000 94.49652534532882
61600000 123.82541580055307
61700000 47.524770742931736
61800000 113.96207152234983
61900000 103.62166169414333
62000000 100.16641263149461
62100000 106.71915584219238
62200000 96.63775751122924
62300000 97.17611223274154
62400000 99.30286112972698
62500000 99.17660096628882
62600000 50.05287981016973
62700000 82.26822569951455
62800000 91.83202380040696
62900000 90.51207578837061
63000000 84.25568596048575
63100000 86.32850036013484
63200000 88.15361431479431
63300000 23.02583116357231
63400000 90.97870705617312
63500000 88.541705785887
63600000 83.5290924511

90300000 78.62659851873552
90400000 43.278234311065916
90500000 59.645672323243005
90600000 74.88620491005558
90700000 75.63895866762284
90800000 77.84068327128041
90900000 20.312896954533628
91000000 79.85743917773041
91100000 75.228906045723
91200000 79.037576827727
91300000 65.48021144891976
91400000 44.111614955225484
91500000 78.1251285839328
91600000 76.90162280875924
91700000 84.72994002832358
91800000 85.56685903484436
91900000 80.4843876594526
92000000 87.08714134279808
92100000 80.84529392795406
92200000 88.3361922173303
92300000 90.86643233969193
92400000 86.1855915244809
92500000 78.54905458761203
92600000 76.73253322883906
92700000 53.82819484323595
92800000 47.365034456081275
92900000 81.47288517175723
93000000 85.49399927775016
93100000 76.77556206817955
93200000 14.277741574436261
93300000 78.67903556162896
93400000 77.51805165909533
93500000 83.97361385476083
93600000 62.124568682398376
93700000 52.6104886055407
93800000 84.54355679033694
93900000 85.69967198537645
940

119900000 48.49865613670903
120000000 62.738267121815085
120100000 63.25631656648796
120200000 68.87060022212292
120300000 73.20108395146246
120400000 77.26880028734024
120500000 33.96949054960809
120600000 29.62685482999941
120700000 43.6223301041835
120800000 10.24090704278094
120900000 44.95333762121093
121000000 29.71876510928065
121100000 35.10975123492492
121200000 52.87084056334883
121300000 64.5350968227896
121400000 41.846352245780864
121500000 41.09107517774606
121600000 44.32333154409079
121700000 55.716981571594346
121800000 47.4739558915545
121900000 47.26835964929439
122000000 63.67993129898413
122100000 42.769759451088355
122200000 54.29947566252307
122300000 62.309668445088306
122400000 37.281051851433844
122500000 40.351066554531634
122600000 55.12100973795027
122700000 43.699743129673955
122800000 35.46277521307068
122900000 18.52090933670867
123000000 37.75044860502505
123100000 10.148294880540073
123200000 38.301397189776154
123300000 22.933869502700926
123400000 38

...

1189900000 3.390531171774192
1190000000 2.9716492768984173
1190100000 3.585576728989273
1190200000 2.988682830634144
1190300000 3.1783559907799632
1190400000 4.030825195799013
1190500000 3.339219101891968
1190600000 2.9415958608499664
1190700000 3.5698534083668094
1190800000 3.281294498620722
1190900000 2.7959800307820872
1191000000 3.308815372326136
1191100000 3.819241204419712
1191200000 3.386509629782859
1191300000 2.86706026288718
1191400000 3.8253696807205677
1191500000 3.7038284291921055
1191600000 4.229477107569794
best so far: 0
type: [1, 4, 1, 3, 11] 132
cases of this type: 1581228
1191700000 2.2658408371256127
1191800000 1.5985559051050808
1191900000 1.3794979940911234
1192000000 1.4891844566072294
1192100000 1.697145400791741
1192200000 1.4680155507257284
1192300000 1.443306703507122
1192400000 2.017960260699302
1192500000 1.4601774560904504
1192600000 1.130621530784789
1192700000 2.0538303169563314
1192800000 1.5776485733224153
1192900000 1.107603967324045
1193000000 1.3422

1203700000 3.382310180543459
1203800000 3.536645670180331
1203900000 3.941525080120822
1204000000 3.4454743352228836
1204100000 2.901929379554
1204200000 4.3793733770607215
1204300000 3.615454618392815
1204400000 3.0643549127772816
1204500000 3.8981477662204207
1204600000 3.346323084345818
1204700000 3.0336463124575945
1204800000 4.17638119015787
1204900000 2.8480032337746555
1205000000 2.897904456910829
1205100000 3.7202219379801553
1205200000 2.6459845725140436
1205300000 3.073272834443586
1205400000 3.9884924926816825
1205500000 2.681002231607228
1205600000 3.2348073119813834
1205700000 4.0310382180369
1205800000 2.0151136166808836
1205900000 3.1709554878467223
1206000000 3.621870307639053
1206100000 2.1933421493077145
1206200000 4.10401518172881
1206300000 3.5675193048352893
1206400000 2.173762050682495
1206500000 3.9180031717215575
1206600000 3.78696151577421
1206700000 2.6671748270121083
1206800000 3.8724133674845267
1206900000 3.5409703825549683
1207000000 3.0803373866287798
120

1230900000 1.1067979681320788
1231000000 1.2976234885367255
1231100000 0.6446495222288867
1231200000 1.218353213439266
1231300000 1.2239500200471647
1231400000 0.7430192366471621
1231500000 1.159882746114224
1231600000 1.20957562191391
1231700000 0.8546276368205613
1231800000 1.266236775015563
1231900000 1.1269355638209515
1232000000 0.7270305908074809
1232100000 1.2586911031241714
1232200000 1.0042286059828964
1232300000 0.8476050362278919
1232400000 1.1667296386935115
1232500000 0.8786866477331488
1232600000 0.8544389562653005
1232700000 1.1705497362185417
1232800000 0.8332973137799998
1232900000 1.1451953185373114
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 1.0237971028218864
1233100000 0.7049384314442608
1233200000 0.9042256823000013
1233300000 1.023747303505808
1233400000 0.6806548829760221
1233500000 0.981445766490476
1233600000 1.0040688379066587
1233700000 0.759467361086405
1233800000 0.9897781500423649
1233900000 0.8540300681783856
12340000

1252500000 0.4219051458465556
1252600000 0.516001111497667
1252700000 0.4240871942112512
1252800000 0.4288165460080604
1252900000 0.47306652044334346
1253000000 0.4144666451340748
1253100000 0.42192931049257515
1253200000 0.4724737070834107
best so far: 0
type: [2, 2, 1, 3, 11] 132
cases of this type: 790614
1253300000 0.3370165044336849
1253400000 0.2758134677039981
1253500000 0.22226186446716387
1253600000 0.2891825448758304
1253700000 0.2616838903472225
1253800000 0.2715140759232475
1253900000 0.24263769229026966
1254000000 0.20116248896244168
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.2362599849394196
1254200000 0.23018647300620543
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.17582661547283332
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.22212185089980893
1254500000 0.13010135782246787
best so far: 0
type: [2, 2, 3, 11, 1] 132
cases of this type: 23958
best so far: 0
type

best so far: 0
type: [3, 2, 11, 1, 2] 132
cases of this type: 3872
best so far: 0
type: [3, 2, 11, 2, 1] 132
cases of this type: 1936
best so far: 0
type: [3, 2, 22, 1, 1] 132
cases of this type: 968
best so far: 0
type: [3, 4, 1, 1, 11] 132
cases of this type: 58564
best so far: 0
type: [3, 4, 1, 11, 1] 132
cases of this type: 5324
best so far: 0
type: [3, 4, 11, 1, 1] 132
cases of this type: 484
best so far: 0
type: [3, 11, 1, 1, 4] 132
cases of this type: 2816
best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 2, 1, 1] 132
cases of this type: 88
best